In [20]:
import pandas as pd
import os
import bs4
import requests

#findout which papers are missing from december
df = pd.read_csv('../FINAL_ARXIV_2025.csv')

#first find the counts per month
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df2 = df.groupby(['month', 'year']).size().reset_index(name='counts')
print(df2)

# visit arxiv.org
url = 'https://arxiv.org/list/astro-ph/2025-12?skip=0&show=2000'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
all_ids = soup.find_all('a', {'title': 'Abstract'})
all_ids = [id.text for id in all_ids]
all_ids = all_ids[:1694]


    month  year  counts
0       1  2025    1344
1       2  2025    1287
2       3  2025    1585
3       4  2025    1467
4       5  2025    1386
5       6  2025    1478
6       7  2025    1816
7       8  2025    1304
8       9  2025    1860
9      10  2025    1889
10     11  2025    1551
11     12  2025    1636


In [24]:
fixed_ids = []
for id in all_ids:
    id =id.strip().lower()
    id = id.replace(':','.org/pdf/')
    fixed_ids.append(id)

print(fixed_ids[0])

arxiv.org/pdf/2512.00054


In [25]:
pdf_links_set = set(df['pdf_link'].str.lower().values)

missing_ids = []
print('arxiv_ids sample:', fixed_ids[0])
print('pdf_link sample:', df['pdf_link'].values[0])
print('Total papers in CSV:', len(df))

for ids in fixed_ids:
    if ids not in pdf_links_set:
        missing_ids.append(ids)

print('Total papers on ArXiv for Dec:', len(fixed_ids))
print('Real missing papers:', len(missing_ids))

arxiv_ids sample: arxiv.org/pdf/2512.00054
pdf_link sample: arxiv.org/pdf/2501.00089
Total papers in CSV: 18603
Total papers on ArXiv for Dec: 1694
Real missing papers: 58


In [26]:
#writing the missing papers to a txt file for extraction
with open('dec_58.txt', 'w') as f:
    for paper in missing_ids:
        f.write(paper + '\n')